# Segmenting and Clustering Neighbourhoods in Toronto

All three Questions andwers are in this same full notebook.
Look in to corresponding section for the related answer.

##  Question 1 :

This notebook was created with Python 3.8.2, Jupyter Lab and TensorFlow GPU kernel.

### First import required libraries

In [1]:
#import all the libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline
print('Packages installed  :)')

Packages installed  :)


#### Get the page from Wikipedia

The page given in excersice has new layout and nested tab/cr split table.

Using the old page with simple table in it. 

In [2]:
# Get the file using beautiful soup 
#url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"

result = requests.get(url)
data_html = BeautifulSoup(result.content)

In [3]:
soup = BeautifulSoup(str(data_html))
neigh = soup.find('table')
table_str = str(neigh.extract())
df = pd.read_html(table_str)[0]
df.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

Drop all the rows which have Boroughs 'Not assigned' and then reset the index. Assumed looking at the wikipedia link that the Neighborhoods with the same Postal Code appear on the same row so further cleansing didnt have to be done to satisfy requirements

In [4]:
df_dropna = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_dropna.rename(columns={'Postcode' : 'PostalCode'}, inplace=True)
print(df_dropna.shape) # Check to see the shape of the dataframe, comment out when funtional

(211, 3)


### Making Neighborhood "Not assigned", same as Borough name:

In [5]:
for index, row in df_dropna.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df_dropna.head()

PostalCode           Borough     Neighbourhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

Preprocess data so that Broroughs are grouped based on their name

In [6]:
df = df_dropna
df.head()

PostalCode           Borough     Neighbourhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

In [7]:
df_grouped = df.groupby(['Borough', 'PostalCode'], as_index=False).agg(lambda x:','.join(x))
df_grouped.head(20)

Borough PostalCode  \
0    Central Toronto        M4N   
1    Central Toronto        M4P   
2    Central Toronto        M4R   
3    Central Toronto        M4S   
4    Central Toronto        M4T   
5    Central Toronto        M4V   
6    Central Toronto        M5N   
7    Central Toronto        M5P   
8    Central Toronto        M5R   
9   Downtown Toronto        M4W   
10  Downtown Toronto        M4X   
11  Downtown Toronto        M4Y   
12  Downtown Toronto        M5A   
13  Downtown Toronto        M5B   
14  Downtown Toronto        M5C   
15  Downtown Toronto        M5E   
16  Downtown Toronto        M5G   
17  Downtown Toronto        M5H   
18  Downtown Toronto        M5J   
19  Downtown Toronto        M5K   

                                        Neighbourhood  
0                                       Lawrence Park  
1                                    Davisville North  
2                                  North Toronto West  
3                                          Davisville  
4                          Moore Park,Summerhill East  
5   Deer Park,Forest Hill SE,Rathnelly,South Hill,...  
6                                            Roselawn  
7                  Forest Hill North,Forest Hill West  
8                   The Annex,North Midtown,Yorkville  
9                                            Rosedale  
10                         Cabbagetown,St. James Town  
11                               Church and Wellesley  
12                           Harbourfront,Regent Park  
13                            Ryerson,Garden District  
14                                     St. James Town  
15                                        Berczy Park  
16                                 Central Bay Street  
17                             Adelaide,King,Richmond  
18    Harbourfront East,Toronto Islands,Union Station  
19            Design Exchange,Toronto Dominion Centre

In [8]:
df_grouped.loc[df_grouped['Borough'].isin(["Not assigned"])] #check ih empty


Empty DataFrame
Columns: [Borough, PostalCode, Neighbourhood]
Index: []

So now all the requirements are met and the data is cleansed, now we just have to add the Latitude and Longitudes of each location

In [9]:
df_grouped.reset_index(drop=True)

Borough PostalCode                                Neighbourhood
0    Central Toronto        M4N                                Lawrence Park
1    Central Toronto        M4P                             Davisville North
2    Central Toronto        M4R                           North Toronto West
3    Central Toronto        M4S                                   Davisville
4    Central Toronto        M4T                   Moore Park,Summerhill East
..               ...        ...                                          ...
98              York        M6C                           Humewood-Cedarvale
99              York        M6E                          Caledonia-Fairbanks
100             York        M6M  Del Ray,Keelesdale,Mount Dennis,Silverthorn
101             York        M6N                 The Junction North,Runnymede
102             York        M9N                                       Weston

[103 rows x 3 columns]

In [10]:
list(df_grouped.columns.values)

['Borough', 'PostalCode', 'Neighbourhood']

Rearranged the Columns so the PostalCode appears first

In [11]:
df_grouped = df_grouped[['PostalCode', 'Borough', 'Neighbourhood']]
list(df_grouped.columns.values)

['PostalCode', 'Borough', 'Neighbourhood']

In [12]:
df_grouped.head()

PostalCode          Borough               Neighbourhood
0        M4N  Central Toronto               Lawrence Park
1        M4P  Central Toronto            Davisville North
2        M4R  Central Toronto          North Toronto West
3        M4S  Central Toronto                  Davisville
4        M4T  Central Toronto  Moore Park,Summerhill East

In [13]:
df_grouped.shape

(103, 3)

## Question 2 :

Now we are going to create a new table with the Latitudes and Longitudes corresponding to the different PostalCodes

In [14]:
#geo_url = "https://cocl.us/Geospatial_data"
geo_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"

geo_df = pd.read_csv(geo_url)
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_df.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

Next We Will merge the data. We will make the actual data equal to the temporary data frame we are working with becauase we are happy with it and then we will merge it based on the postal code.

In [15]:
df = df_grouped
df = pd.merge(df, geo_df, on='PostalCode')
df.head()


PostalCode          Borough               Neighbourhood   Latitude  \
0        M4N  Central Toronto               Lawrence Park  43.728020   
1        M4P  Central Toronto            Davisville North  43.712751   
2        M4R  Central Toronto          North Toronto West  43.715383   
3        M4S  Central Toronto                  Davisville  43.704324   
4        M4T  Central Toronto  Moore Park,Summerhill East  43.689574   

   Longitude  
0 -79.388790  
1 -79.390197  
2 -79.405678  
3 -79.388790  
4 -79.383160

## Question 3 :

### Map of Toronto

Creating map of Toronto using latitude and longitude values and adding markers to it.

In [16]:
# create map of Toronto using latitude and longitude values
latitude = 43.653225
longitude = -79.383186
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.75,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


### Next we will define foursquare Credentials

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20200514' # user api version date
LIMIT = 100 # limit of number of venues returned by Foursquare API
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VIHECMBTSH4ODEFH0RIRIAYH3BSUFEDYVA12GYARKYEG331N
CLIENT_SECRET:VBMBP4TNVAWCV5XO55YC23QOUTD5LLGTQBLR4ZGXD2WRPZDK


In [18]:
df.loc[0, 'Neighbourhood']

'Lawrence Park'

We will analyze the first Neighborhood, Lawrence Park

In [19]:
law_lat = df.loc[0, 'Latitude']
law_long = df.loc[0, 'Longitude']
law_name = df.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(law_name, law_lat, law_long))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.


In [20]:
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    law_lat, 
    law_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=VIHECMBTSH4ODEFH0RIRIAYH3BSUFEDYVA12GYARKYEG331N&client_secret=VBMBP4TNVAWCV5XO55YC23QOUTD5LLGTQBLR4ZGXD2WRPZDK&v=20200514&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '608e6978f365de05478172b7'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

Firs define function that extracts the category of the venue

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Next we will clean the json and structure it into a pandas dataframe

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


name   categories        lat        lng
0            Lawrence Park Ravine         Park  43.726963 -79.394382
1              Zodiac Swim School  Swim School  43.728532 -79.382860
2  TTC Bus #162 - Lawrence-Donway     Bus Line  43.728026 -79.382805

Let's look how many venues around Lawrence Avenue were found

In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


That was just one Neighborhood in Toronto.
For all venues we will use a forloop to run through all the neighborhoods in Toronto repeating above process.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



Now we will run the above function on each neighborhood and create a new dataframe called toronto_venues

In [26]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'], longitudes=df['Longitude']
)


Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Woodbine Gardens,Parkview Hill
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Humber 

In [27]:
print('Venues table shape: ',toronto_venues.shape)
print(' ')
print(toronto_venues.head())

Venues table shape:  (2132, 7)
 
       Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0     Lawrence Park              43.728020              -79.388790   
1     Lawrence Park              43.728020              -79.388790   
2     Lawrence Park              43.728020              -79.388790   
3  Davisville North              43.712751              -79.390197   
4  Davisville North              43.712751              -79.390197   

                            Venue  Venue Latitude  Venue Longitude  \
0            Lawrence Park Ravine       43.726963       -79.394382   
1              Zodiac Swim School       43.728532       -79.382860   
2  TTC Bus #162 - Lawrence-Donway       43.728026       -79.382805   
3         Summerhill Market North       43.715499       -79.392881   
4                   Sherwood Park       43.716551       -79.387776   

      Venue Category  
0               Park  
1        Swim School  
2           Bus Line  
3  Food & Drink Shop  
4         

list how many venues were returned for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                                 95   
Agincourt                                                               4   
Agincourt North,L'Amoreaux East,Milliken,Steele...                      3   
Albion Gardens,Beaumond Heights,Humbergate,Jame...                      9   
Alderwood,Long Branch                                                   6   
...                                                                   ...   
Willowdale West                                                         5   
Woburn                                                                  4   
Woodbine Gardens,Parkview Hill                                         11   
Woodbine Heights                                                        9   
York Mills West                                                         3   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide,King,Richmond                                                  95   
Agincourt                                                                4   
Agincourt North,L'Amoreaux East,Milliken,Steele...                       3   
Albion Gardens,Beaumond Heights,Humbergate,Jame...                       9   
Alderwood,Long Branch                                                    6   
...                                                                    ...   
Willowdale West                                                          5   
Woburn                                                                   4   
Woodbine Gardens,Parkview Hill                                          11   
Woodbine Heights                                                         9   
York Mills West                                                          3   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                 95              95   
Agincourt                                               4               4   
Agincourt North,L'Amoreaux East,Milliken,Steele...      3               3   
Albion Gardens,Beaumond Heights,Humbergate,Jame...      9               9   
Alderwood,Long Branch                                   6               6   
...                                                   ...             ...   
Willowdale West                                         5               5   
Woburn                                                  4               4   
Woodbine Gardens,Parkview Hill                         11              11   
Woodbine Heights                                        9               9   
York Mills West                                         3               3   

                                                    Venue Longitude  \
Neighborhood                                                          
Adelaide,King,Richmond                                           95   
Agincourt                                                         4   
Agincourt North,L'Amoreaux East,Milliken,Steele...                3   
Albion Gardens,Beaumond Heights,Humbergate,Jame...                9   
Alderwood,Long Branch                                             6   
...                                                             ...   
Willowdale West                                                   5   
Woburn                                                            4   
Woodbine Gardens,Parkview Hill                                   11   
Woodbine Heights                                                  9   
York Mills West                                                   3   

                                                    Venue Category  
Neighborhood                            

Check how many unique Venue categories

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


## Next analyze each neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print('Shape of dataframe', toronto_onehot.shape)
print(' ')
print(toronto_onehot.head())


Shape of dataframe (2132, 271)
 
   Yoga Studio  Accessories Store  Adult Boutique  Airport  \
0            0                  0               0        0   
1            0                  0               0        0   
2            0                  0               0        0   
3            0                  0               0        0   
4            0                  0               0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  ...  Truck Stop  \
0                    0             0  ...           0   
1                    0 

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                              Adelaide,King,Richmond          0.0   
1                                           Agincourt          0.0   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...          0.0   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...          0.0   
4                               Alderwood,Long Branch          0.0   
..                                                ...          ...   
93                                    Willowdale West          0.0   
94                                             Woburn          0.0   
95                     Woodbine Gardens,Parkview Hill          0.0   
96                                   Woodbine Heights          0.0   
97                                    York Mills West          0.0   

    Accessories Store  Adult Boutique  Airport  Airport Food Court  \
0                 0.0             0.0      0.0                 0.0   
1                 0.0             0.0      0.0                 0.0   
2                 0.0             0.0      0.0                 0.0   
3                 0.0             0.0      0.0                 0.0   
4                 0.0             0.0      0.0                 0.0   
..                ...             ...      ...                 ...   
93                0.0             0.0      0.0                 0.0   
94                0.0             0.0      0.0                 0.0   
95                0.0             0.0      0.0                 0.0   
96                0.0             0.0      0.0                 0.0   
97                0.0             0.0      0.0                 0.0   

    Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0              0.0              0.0               0.0             0.021053   
1              0.0              0.0               0.0             0.000000   
2              0.0              0.0               0.0             0.000000   
3              0.0              0.0               0.0             0.000000   
4              0.0              0.0               0.0             0.000000   
..             ...              ...               ...                  ...   
93             0.0              0.0               0.0             0.000000   
94             0.0              0.0               0.0             0.000000   
95             0.0              0.0               0.0             0.000000   
96             0.0              0.0               0.0             0.000000   
97             0.0              0.0               0.0             0.000000   

    ...  Truck Stop  Vegetarian / Vegan Restaurant  Video Game Store  \
0   ...         0.0                       0.010526               0.0   
1   ...         0.0                       0.000000               0.0   
2   ...         0.0                       0.000000               0.0   
3   ...         0.0                       0.000000               0.0   
4   ...         0.0                       0.000000               0.0   
..  ...         ...                            ...               ...   
93  ...         0.0                       0.000000               0.0   
94  ...         0.0                       0.000000               0.0   
95  ...         0.0                       0.000000               0.0   
96  ...         0.0                       0.000000               0.0   
97  ...         0.0                       0.000000               0.0   

    Video Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  \
0      0.000000                    0.0              0.0       0.0        0.0   
1      0.000000                    0.0              0.0       0.0        0.0   
2      0.000000                    0.0              0.0       0.0        0.0   
3      0.000000                    0.0              0.0       0.0        0.0   
4      0.000000                    0.0              0.0       0.0        0.0   
..          ...                    ...              ...       ...        ... 

Print each neighborhood with the five most common venues

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.11
1             Café  0.05
2       Restaurant  0.04
3  Thai Restaurant  0.04
4    Deli / Bodega  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4  Middle Eastern Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0         Intersection  0.33
1           Playground  0.33
2                 Park  0.33
3   Mexican Restaurant  0.00
4  Monument / Landmark  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
            venue  freq
0     Pizza Place  0.22
1   Grocery Store  0.22
2  Sandwich Place  0.11
3        Pharmacy  0.11
4      Beer Store  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.33
1             

Let's put that into a pandas dataframe

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Next list the top 10 venues for each of the neighborhood

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(11)

Neighborhood 1st Most Common Venue  \
0                              Adelaide,King,Richmond           Coffee Shop   
1                                           Agincourt                Lounge   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...          Intersection   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...           Pizza Place   
4                               Alderwood,Long Branch           Pizza Place   
5       Bathurst Manor,Downsview North,Wilson Heights           Coffee Shop   
6                                     Bayview Village                  Café   
7                    Bedford Park,Lawrence Manor East    Italian Restaurant   
8                                         Berczy Park           Coffee Shop   
9                          Birch Cliff,Cliffside West       College Stadium   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...           Pizza Place   

        2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
0                        Café             Restaurant       Thai Restaurant   
1   Latin American Restaurant         Breakfast Spot        Clothing Store   
2                  Playground                   Park    Mexican Restaurant   
3               Grocery Store         Sandwich Place              Pharmacy   
4                         Pub         Sandwich Place                   Gym   
5                        Bank                   Park         Shopping Mall   
6         Japanese Restaurant                   Bank    Chinese Restaurant   
7              Sandwich Place             Restaurant           Coffee Shop   
8                Cocktail Bar                 Bakery           Cheese Shop   
9                Skating Rink  General Entertainment                  Café   
10               Liquor Store             Beer Store           Coffee Shop   

        5th Most Common Venue            6th Most Common Venue  \
0               Deli / Bodega                   Clothing Store   
1   Middle Eastern Restaurant              Monument / Landmark   
2         Monument / Landmark  Molecular Gastronomy Restaurant   
3                  Beer Store              Fried Chicken Joint   
4                 Coffee Shop              Monument / Landmark   
5   Middle Eastern Restaurant                Mobile Phone Shop   
6                 Yoga Studio              Moroccan Restaurant   
7           Indian Restaurant             Fast Food Restaurant   
8              Farmers Market               Seafood Restaurant   
9                        Farm                   Massage Studio   
10                       Café                Convenience Store   

              7th Most Common Venue            8th Most Common Venue  \
0                               Gym               Seafood Restaurant   
1   Molecular Gastronomy Restaurant       Modern European Restaurant   
2        Modern European Restaurant                Mobile Phone Shop   
3              Fast Food Restaurant                  Organic Grocery   
4   Molecular Gastronomy Restaurant       Modern European Restaurant   
5                    Sandwich Place                      Supermarket   
6               Monument / Landmark  Molecular Gastronomy Restaurant   
7                      Liquor Store                          Butcher   
8                          Beer Bar                         Pharmacy   
9                    Medical Center         Mediterranean Restaurant   
10                        Pet Store                         Pharmacy   

         9th Most Common Venue     10th Most Common Venue  
0                  Salad Place               Concert Hall  
1            Mobile Phone Shop         Miscellaneous Shop  
2           Miscellaneous Shop  Middle Eastern Restaurant  
3                 Optical Shop                  Pet Store  
4            Mobile Phone Shop         Miscellaneous Shop  
5          Fried Chicken Joint           Sushi Restaurant  
6   Modern European Restaurant          Mobile Phone Shop  
7                

Cluster these Neighbourhoods

In [35]:
from sklearn.cluster import KMeans
kclusters = 5 # set number of clusters

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood. Let's first try the Elbow method to find the optimum K value.


In [36]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=15, random_state=8)
X = toronto_grouped.drop(['Neighborhood'], axis=1)

kmeans.fit(X)
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [37]:
def get_inertia(n_clusters):
    km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=15, random_state=8)
    km.fit(X)
    return km.inertia_

In [38]:
scores = [get_inertia(x) for x in range(2, 21)]


In [39]:
plt.figure(figsize=[10, 8])
sns.lineplot(x=range(2, 21), y=scores)
plt.title("K vs Error")
plt.xticks(range(2, 21))
plt.xlabel("K")
plt.ylabel("Error")

Text(0, 0.5, 'Error')

We can see from above that the point that sharply turns and reduces the speed of the error is ad k=5 (in this dataset)

In [40]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

PostalCode          Borough               Neighbourhood   Latitude  \
0        M4N  Central Toronto               Lawrence Park  43.728020   
1        M4P  Central Toronto            Davisville North  43.712751   
2        M4R  Central Toronto          North Toronto West  43.715383   
3        M4S  Central Toronto                  Davisville  43.704324   
4        M4T  Central Toronto  Moore Park,Summerhill East  43.689574   

   Longitude  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.388790             1.0                  Park              Bus Line   
1 -79.390197             0.0           Pizza Place        Breakfast Spot   
2 -79.405678             0.0           Coffee Shop        Clothing Store   
3 -79.388790             0.0          Dessert Shop        Sandwich Place   
4 -79.383160             2.0            Restaurant           Yoga Studio   

  3rd Most Common Venue            4th Most Common Venue  \
0           Swim School                      Yoga Studio   
1      Department Store                Food & Drink Shop   
2   Sporting Goods Shop                      Yoga Studio   
3                  Café                 Sushi Restaurant   
4    Mexican Restaurant  Molecular Gastronomy Restaurant   

        5th Most Common Venue       6th Most Common Venue  \
0          Mexican Restaurant  Modern European Restaurant   
1                        Park                       Hotel   
2          Italian Restaurant                        Park   
3                 Pizza Place                 Coffee Shop   
4  Modern European Restaurant           Mobile Phone Shop   

  7th Most Common Venue      8th Most Common Venue      9th Most Common Venue  \
0     Mobile Phone Shop         Miscellaneous Shop  Middle Eastern Restaurant   
1                   Gym       Gym / Fitness Center             Sandwich Place   
2            Restaurant         Chinese Restaurant                      Diner   
3                   Gym         Italian Restaurant          Indian Restaurant   
4    Miscellaneous Shop  Middle Eastern Restaurant                Men's Store   

  10th Most Common Venue  
0            Men's Store  
1         Medical Center  
2     Salon / Barbershop  
3             Restaurant  
4    Moroccan Restaurant

In [42]:
toronto_merged.tail()

PostalCode Borough                                Neighbourhood  \
98         M6C    York                           Humewood-Cedarvale   
99         M6E    York                          Caledonia-Fairbanks   
100        M6M    York  Del Ray,Keelesdale,Mount Dennis,Silverthorn   
101        M6N    York                 The Junction North,Runnymede   
102        M9N    York                                       Weston   

      Latitude  Longitude  Cluster_Labels 1st Most Common Venue  \
98   43.693781 -79.428191             1.0                 Field   
99   43.689026 -79.453512             1.0                  Park   
100  43.691116 -79.476013             0.0  Fast Food Restaurant   
101  43.673185 -79.487262             0.0  Caribbean Restaurant   
102  43.706876 -79.518188             1.0         Jewelry Store   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
98                   Park          Hockey Arena                 Trail   
99          Women's Store                  Pool           Men's Store   
100            Restaurant        Discount Store   Fried Chicken Joint   
101        Breakfast Spot         Grocery Store               Brewery   
102     Convenience Store           Yoga Studio    Mexican Restaurant   

               5th Most Common Venue       6th Most Common Venue  \
98                       Yoga Studio          Mexican Restaurant   
99   Molecular Gastronomy Restaurant  Modern European Restaurant   
100                   Sandwich Place              Medical Center   
101               Miscellaneous Shop         Moroccan Restaurant   
102  Molecular Gastronomy Restaurant  Modern European Restaurant   

               7th Most Common Venue            8th Most Common Venue  \
98   Molecular Gastronomy Restaurant       Modern European Restaurant   
99                 Mobile Phone Shop               Miscellaneous Shop   
100         Mediterranean Restaurant                      Men's Store   
101              Monument / Landmark  Molecular Gastronomy Restaurant   
102                Mobile Phone Shop               Miscellaneous Shop   

          9th Most Common Venue 10th Most Common Venue  
98            Mobile Phone Shop     Miscellaneous Shop  
99    Middle Eastern Restaurant     Mexican Restaurant  
100          Mexican Restaurant    Monument / Landmark  
101  Modern European Restaurant      Mobile Phone Shop  
102   Middle Eastern Restaurant            Men's Store

Drop invalid values in Clusters or Venues

In [43]:
#toronto_drop = toronto_merged[toronto_merged.Cluster_Labels != 'NaN'].reset_index(drop=True)
toronto_merged.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)


### Visualize

Here we see where are most of the venues located. This gives us indication where most of the people are going around and therefore best places for hospitality and retail businesses.

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.75).add_to(map_clusters)
       
map_clusters


## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can assign a name to each cluster.


### Cluster 1

In [45]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Borough  Cluster_Labels    1st Most Common Venue  \
1    Central Toronto             0.0              Pizza Place   
2    Central Toronto             0.0              Coffee Shop   
3    Central Toronto             0.0             Dessert Shop   
5    Central Toronto             0.0              Coffee Shop   
6    Central Toronto             0.0  Health & Beauty Service   
..               ...             ...                      ...   
95      West Toronto             0.0       Mexican Restaurant   
96      West Toronto             0.0           Breakfast Spot   
97      West Toronto             0.0              Coffee Shop   
100             York             0.0     Fast Food Restaurant   
101             York             0.0     Caribbean Restaurant   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1          Breakfast Spot      Department Store     Food & Drink Shop   
2          Clothing Store   Sporting Goods Shop           Yoga Studio   
3          Sandwich Place                  Café      Sushi Restaurant   
5      Light Rail Station            Bagel Shop                   Pub   
6                  Garden           Music Venue           Yoga Studio   
..                    ...                   ...                   ...   
95        Thai Restaurant                  Café        Discount Store   
96              Gift Shop           Coffee Shop                  Bank   
97                   Café                   Pub           Pizza Place   
100            Restaurant        Discount Store   Fried Chicken Joint   
101        Breakfast Spot         Grocery Store               Brewery   

         5th Most Common Venue            6th Most Common Venue  \
1                         Park                            Hotel   
2           Italian Restaurant                             Park   
3                  Pizza Place                      Coffee Shop   
5                   Restaurant                     Liquor Store   
6    Middle Eastern Restaurant  Molecular Gastronomy Restaurant   
..                         ...                              ...   
95         Arts & Crafts Store                        Gastropub   
96                     Dog Run      Eastern European Restaurant   
97          Italian Restaurant                 Sushi Restaurant   
100             Sandwich Place                   Medical Center   
101         Miscellaneous Shop              Moroccan Restaurant   

          7th Most Common Venue            8th Most Common Venue  \
1                           Gym             Gym / Fitness Center   
2                    Restaurant               Chinese Restaurant   
3                           Gym               Italian Restaurant   
5                   Supermarket                 Sushi Restaurant   
6    Modern European Restaurant                Mobile Phone Shop   
..                          ...                              ...   
95                          Bar              Fried Chicken Joint   
96           Italian Restaurant                       Restaurant   
97                  Yoga Studio               Falafel Restaurant   
100    Mediterranean Restaurant                      Men's Store   
101         Monument / Landmark  Molecular Gastronomy Restaurant   

          9th Most Common Venue 10th Most Common Venue  
1                Sandwich Place         Medical Center  
2                         Diner     Salon / Barbershop  
3             Indian Restaurant             Restaurant  
5           Fried Chicken Joint                   Bank  
6            Miscellaneous Shop     Mexican Restaurant  
..                          ...                    ...  
95                       Bakery            Music Venue  
96                Movie Theater           Dessert Shop  
97            Fish & Chips Shop             Restaurant  
100          Mexican Restaurant    Monument / Landmark  
101  Modern European Restaurant      Mobile Phone Shop  

[74 rows x 12 columns]

### Cluster 2 

In [46]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



Borough  Cluster_Labels   1st Most Common Venue  \
0     Central Toronto             1.0                    Park   
7     Central Toronto             1.0           Jewelry Store   
9    Downtown Toronto             1.0                    Park   
33          East York             1.0            Intersection   
36          East York             1.0                    Park   
46          Etobicoke             1.0                    Park   
56         North York             1.0                    Park   
58         North York             1.0       Food & Drink Shop   
63         North York             1.0                 Airport   
64         North York             1.0           Grocery Store   
70         North York             1.0                  Bakery   
71         North York             1.0                  Bakery   
72         North York             1.0  Furniture / Home Store   
76        Scarborough             1.0                     Bar   
82        Scarborough             1.0                  Bakery   
89        Scarborough             1.0            Intersection   
98               York             1.0                   Field   
99               York             1.0                    Park   
102              York             1.0           Jewelry Store   

          2nd Most Common Venue      3rd Most Common Venue  \
0                      Bus Line                Swim School   
7                          Park                      Trail   
9                    Playground                      Trail   
33                  Video Store                   Bus Stop   
36            Convenience Store               Intersection   
46            Mobile Phone Shop             Sandwich Place   
56   Construction & Landscaping          Convenience Store   
58                         Park                Yoga Studio   
63                     Bus Stop                       Park   
64                         Bank                       Park   
70          Japanese Restaurant                       Park   
71   Construction & Landscaping                       Park   
72                 Intersection  Middle Eastern Restaurant   
76                  Yoga Studio  Middle Eastern Restaurant   
82                  Bus Station               Soccer Field   
89                   Playground                       Park   
98                         Park               Hockey Arena   
99                Women's Store                       Pool   
102           Convenience Store                Yoga Studio   

          4th Most Common Venue            5th Most Common Venue  \
0                   Yoga Studio               Mexican Restaurant   
7              Sushi Restaurant                      Yoga Studio   
9                   Yoga Studio                      Men's Store   
33           Athletics & Sports                     Dance Studio   
36                  Yoga Studio        Middle Eastern Restaurant   
46                  Yoga Studio               Mexican Restaurant   
56                  Yoga Studio               Mexican Restaurant   
58   Modern European Restaurant                Mobile Phone Shop   
63                  Yoga Studio               Mexican Restaurant   
64                Shopping Mall                      Yoga Studio   
70             Asian Restaurant                      Yoga Studio   
71             Basketball Court                            Trail   
72          Monument / Landmark  Molecular Gastronomy Restaurant   
76          Monument / Landmark  Molecular Gastronomy Restaurant   
82                 Intersection                             Park   
89           Mexican Restaurant              Monument / Landmark   
98                        Trail                      Yoga Studio   
99                  Men's Store  Molecular Gastronomy Restaurant   
102          Mexican Restaurant  Molecular Gastronomy Restaurant   

               6th Most Common Venue            7th Most Common Venue  \
0         Modern European Restaurant  

## Cluster 3

In [47]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
4  Central Toronto             2.0            Restaurant   

  2nd Most Common Venue 3rd Most Common Venue  \
4           Yoga Studio    Mexican Restaurant   

             4th Most Common Venue       5th Most Common Venue  \
4  Molecular Gastronomy Restaurant  Modern European Restaurant   

  6th Most Common Venue 7th Most Common Venue      8th Most Common Venue  \
4     Mobile Phone Shop    Miscellaneous Shop  Middle Eastern Restaurant   

  9th Most Common Venue 10th Most Common Venue  
4           Men's Store    Moroccan Restaurant

## Cluster 4

In [48]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue 2nd Most Common Venue  \
40   Etobicoke             3.0                  Park        Baseball Field   
65  North York             3.0            Food Truck      Business Service   
73  North York             3.0        Baseball Field           Yoga Studio   

        3rd Most Common Venue 4th Most Common Venue  \
40                Yoga Studio    Mexican Restaurant   
65             Baseball Field     Korean Restaurant   
73  Middle Eastern Restaurant   Monument / Landmark   

              5th Most Common Venue            6th Most Common Venue  \
40  Molecular Gastronomy Restaurant       Modern European Restaurant   
65                      Yoga Studio  Molecular Gastronomy Restaurant   
73  Molecular Gastronomy Restaurant       Modern European Restaurant   

         7th Most Common Venue 8th Most Common Venue  \
40           Mobile Phone Shop    Miscellaneous Shop   
65  Modern European Restaurant     Mobile Phone Shop   
73           Mobile Phone Shop    Miscellaneous Shop   

        9th Most Common Venue     10th Most Common Venue  
40  Middle Eastern Restaurant                Men's Store  
65         Miscellaneous Shop  Middle Eastern Restaurant  
73         Mexican Restaurant                     Lounge

## Cluster 5

In [49]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Empty DataFrame
Columns: [Borough, Cluster_Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

## Cluster 6

In [50]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Empty DataFrame
Columns: [Borough, Cluster_Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []

## Conclusion :

Venues that are in Lawrence Park within a radius of 500 meters most are in Clusters 1 and 2. There are only one boroughs in Clusters 5 and 6.

We can select other centerpoints and distance parameters to examine other interesting locations. This can be used for example to get information what happens around particular hotel location and submitted to customers on daily bases or used by travel agent for hotel recommendations.
